## PRÁCTICA ELASTIC
### Alejandro Manuel López Gómez

En primer lugar se realiza el mapeo de los datos del tráfico de madrid

In [1]:
import requests
import utm
import json
import requests
from datetime import datetime
from xml.etree import ElementTree
from unidecode import unidecode

response = requests.get("https://informo.madrid.es/informo/tmadrid/pm.xml")
root = ElementTree.fromstring(response.content)

# Extracción fecha
fecha = datetime.strptime(root[0].text, '%d/%m/%Y %H:%M:%S')

# Almacenar medidas en diccionario y añadir a lista out tras formateo
out = []
for pm in root[1:-1]:
    d={}
    for i in pm:
        try:
            if i.tag == 'intensidad':
                d[i.tag] = int(i.text)
            elif i.tag == 'ocupacion':
                d[i.tag] = int(i.text)
            elif i.tag == 'carga':
                d[i.tag] = int(i.text)
            elif i.tag == 'nivelServicio':
                d[i.tag] = int(i.text)
            elif i.tag == 'intensidadSat':
                d[i.tag] = int(i.text)
            elif i.tag == 'descripcion':
                d[i.tag] = i.text.replace("Ø","n.").replace("`","").replace("'","")
            elif i.tag == 'st_x':
                st_x = i.text.replace(",",".")
                del d[i.tag]
            elif i.tag == 'st_y':
                st_y = i.text.replace(",",".")
                c = utm.to_latlon(float(st_x),float(st_y),30,"T")
                d['coordenadas'] = {'lat' : c[0], 'lon' : c[1]}
                del d[i.tag]
            else:
                d[i.tag] = i.text
        except TypeError:
            pass
        except KeyError:
            pass
        except:
            del d[i.tag]
        
    out.append(d)

Primera prueba: Ingesta de datos con POST

In [5]:
import time

url = "https://despliegue-prueba-ssdd.es.us-central1.gcp.cloud.es.io:9243/"
user = "elastic"
passwd = "ZSm6L8mocqryCuNcWvHug88v"
index = "trafico-madrid"
headers = {"Content-Type" : "application/json"}
api = """/_doc"""

start = time.time()
for i in out:
    response = requests.post(url+index+api, data=json.dumps(i), headers=headers, auth=(user,passwd))

end = time.time()
print("Elapsed time: " + str(end-start))

# Tiempo total: 3027.983315706253 segundos o 50.4663886 minutos

Elapsed time: 3027.983315706253


Segunda prueba: Empleando el API Bulk

In [6]:
import time

# Crear objeto ndjson
out_str = [json.dumps(i) for i in out]
ndjson = '{"index":{"_index":"trafico-madrid"}}\n'+'\n{"index":{"_index":"trafico-madrid"}}\n'.join(out_str)+'\n'

#Api Bulk
url = "https://despliegue-prueba-ssdd.es.us-central1.gcp.cloud.es.io:9243/"
user = "elastic"
passwd = "ZSm6L8mocqryCuNcWvHug88v"
index = "trafico-madrid"
headers = {"Content-Type" : "application/x-ndjson"}
api = """/_bulk"""

start = time.time()
response = requests.post(url+index+api, data=ndjson, headers=headers, auth=(user,passwd))
end = time.time()
print("Response:", response.status_code, response.text)
print("Elapsed time: " + str(end-start))

# Tiempo total: 4 segundos

Response: 200 {"took":150,"errors":false,"items":[{"index":{"_index":"trafico-madrid","_id":"MWTDdIcBAEflflAJgFVf","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4649,"_primary_term":1,"status":201}},{"index":{"_index":"trafico-madrid","_id":"MmTDdIcBAEflflAJgFVf","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4650,"_primary_term":1,"status":201}},{"index":{"_index":"trafico-madrid","_id":"M2TDdIcBAEflflAJgFVf","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4651,"_primary_term":1,"status":201}},{"index":{"_index":"trafico-madrid","_id":"NGTDdIcBAEflflAJgFVf","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4652,"_primary_term":1,"status":201}},{"index":{"_index":"trafico-madrid","_id":"NWTDdIcBAEflflAJgFVf","_version":1,"result":"created","_shards":{"total":2,"successful":2,"failed":0},"_seq_no":4547,"_primary_term":1,